##### Machine Learning Course Project
# Preprocessing
##### Darryl Abraham, Riccardo Paciello

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import operator
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from scipy.stats import chi2_contingency, fisher_exact
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import mutual_info_classif
import prince
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

### Loading the Data

In [2]:
df = pd.read_csv('./data/cup98LRN.txt', sep = ',', chunksize=10000, low_memory=False)
df = pd.concat(df, axis='rows')
df.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [3]:
def get_feature_types(df):
    feature_types = {}
    for col in df.columns:
        feature_types[col] = df[col].dtype
    return feature_types
feature_types = get_feature_types(df)

In [4]:
df.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [5]:
id = ['CONTROLN']
num_feats = ['ODATEDW', 'DOB', 'AGE', 'NUMCHLD', 'INCOME', 'HIT', 'MBCRAFT', "MBGARDEN", "MBBOOKS", "MBCOLECT", "MAGFAML", "MAGFEM", "MAGMALE", "PUBGARDN", "PUBCULIN", "PUBHLTH", "PUBDOITY", "PUBNEWFN", "PUBPHOTO", "PUBOPP", 'MALEMILI', 'MALEVET', 'VIETVETS', 'WWIIVETS', 'LOCALGOV', 'STATEGOV', 'FEDGOV', 'POP901', 'POP902', 'POP903', 'POP90C1', 'POP90C2', 'POP90C3', 'POP90C4', 'POP90C5', 'ETH1', 'ETH2', 'ETH3', 'ETH4', 'ETH5', 'ETH6', 'ETH7', 'ETH8', 'ETH9', 'ETH10', 'ETH11', 'ETH12', 'ETH13', 'ETH14', 'ETH15', 'ETH16', 'AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGE907', 'CHIL1', 'CHIL2', 'CHIL3', 'AGEC1', 'AGEC2', 'AGEC3', 'AGEC4', 'AGEC5', 'AGEC6', 'AGEC7', 'CHILC1', 'CHILC2', 'CHILC3', 'CHILC4', 'CHILC5', "HHAGE1", "HHAGE2", "HHAGE3", "HHN1", "HHN2", "HHN3", "HHN4", "HHN5", "HHN6", "MARR1", "MARR2", "MARR3", "MARR4", "HHP1", "HHP2", "DW1", "DW2", "DW3", "DW4", "DW5", "DW6", "DW7", "DW8", "DW9", "HV1", "HV2", "HV3", "HV4", "HU1", "HU2", "HU3", "HU4", "HU5", "HHD1", "HHD2", "HHD3", "HHD4", "HHD5", "HHD6", "HHD7", "HHD8", "HHD9", "HHD10", "HHD11", "HHD12", "ETHC1", "ETHC2", "ETHC3", "ETHC4","ETHC5", "ETHC6", "HVP1", "HVP2", "HVP3", "HVP4", "HVP5", "HVP6", "HUR1", "HUR2", "RHP1", "RHP2", "RHP3", "RHP4", "HUPA1", "HUPA2", "HUPA3", "HUPA4", "HUPA5", "HUPA6", "HUPA7", "RP1", "RP2", "RP3", "RP4", "IC1", "IC2", "IC3", "IC4", "IC5", "IC6", "IC7", "IC8", "IC9", "IC10", "IC11", "IC12", "IC13", "IC14", "IC15", "IC16", "IC17", "IC18", "IC19", "IC20", "IC21", "IC22", "IC23", "HHAS1", "HHAS2", "HHAS3", "HHAS4", "MC1", "MC2", "MC3", "TPE1", "TPE2", "TPE3", "TPE4", "TPE5", "TPE6", "TPE7", "TPE8", "TPE9", "PEC1", "PEC2", "TPE10", "TPE11", "TPE12", "TPE13", "LFC1", "LFC2", "LFC3", "LFC4", "LFC5", "LFC6", "LFC7", "LFC8", "LFC9", "LFC10", "OCC1", "OCC2", "OCC3", "OCC4", "OCC5", "OCC6", "OCC7", "OCC8", "OCC9", "OCC10", "OCC11", "OCC12", "OCC13", "EIC1", "EIC2", "EIC3", "EIC4", "EIC5", "EIC6", "EIC7", "EIC8", "EIC9", "EIC10", "EIC11", "EIC12", "EIC13", "EIC14", "EIC15", "EIC16", "OEDC1", "OEDC2", "OEDC3", "OEDC4", "OEDC5", "OEDC6", "OEDC7", "EC1", "EC2", "EC3", "EC4", "EC5", "EC6", "EC7", "EC8", "SEC1", "SEC2", "SEC3", "SEC4", "SEC5", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6", "VC1", "VC2", "VC3", "VC4", "ANC1", "ANC2", "ANC3", "ANC4", "ANC5", "ANC6", "ANC7", "ANC8", "ANC9", "ANC10", "ANC11", "ANC12", "ANC13", "ANC14", "ANC15", "POBC1", "POBC2", "LSC1", "LSC2", "LSC3", "LSC4", "VOC1", "VOC2", "VOC3", "HC1", "HC2", "HC3", "HC4", "HC5", "HC6", "HC7", "HC8", "HC9", "HC10", "HC11", "HC12", "HC13", "HC14", "HC15", "HC16", "HC17", "HC18", "HC19", "HC20", "HC21", "MHUC1", "MHUC2", "AC1", "AC2", 'CARDPROM', 'NUMPROM', 'CARDPM12', 'NUMPRM12', "RAMNT_3", "RAMNT_4", "RAMNT_5", "RAMNT_6", "RAMNT_7", "RAMNT_8", "RAMNT_9", "RAMNT_10", "RAMNT_11", "RAMNT_12", "RAMNT_13", "RAMNT_14", "RAMNT_15", "RAMNT_16", "RAMNT_17", "RAMNT_18", "RAMNT_19", "RAMNT_20", "RAMNT_21", "RAMNT_22", "RAMNT_23", "RAMNT_24", "RAMNTALL", "NGIFTALL", "CARDGIFT", "MINRAMNT", "MINRDATE", "MAXRAMNT", "MAXRDATE", "LASTGIFT", "LASTDATE", "FISTDATE", "NEXTDATE", "TIMELAG", "AVGGIFT", "ADATE_2", "ADATE_3", "ADATE_4", "ADATE_5", "ADATE_6", "ADATE_7", "ADATE_8", "ADATE_9", "ADATE_10", "ADATE_11", "ADATE_12", "ADATE_13", "ADATE_14", "ADATE_15", "ADATE_16", "ADATE_17", "ADATE_18", "ADATE_19", "ADATE_20", "ADATE_21", "ADATE_22", "ADATE_23", "ADATE_24"]
cat_feats = ['OSOURCE', 'TCODE', 'STATE', 'ZIP', 'MAILCODE', 'PVASTATE', 'NOEXCH', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'MDMAUD', 'DOMAIN', 'CLUSTER', 'AGEFLAG', 'HOMEOWNR', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'GENDER', 'WEALTH1', 'DATASRCE', 'SOLP3', 'SOLIH', 'MAJOR', 'WEALTH2', 'GEOCODE', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'LIFESRC', 'PEPSTRFL', "MSA", "ADI", "DMA", "RFA_2", "RFA_3", "RFA_4", "RFA_5", "RFA_6", "RFA_7", "RFA_8", "RFA_9", "RFA_10", "RFA_11", "RFA_12", "RFA_13", "RFA_14", "RFA_15", "RFA_16", "RFA_17", "RFA_18", "RFA_19", "RFA_20", "RFA_21", "RFA_22", "RFA_23", "RFA_24", 'MAXADATE', 'HPHONE_D', "RFA_2R", "RFA_2F", "RFA_2A", "MDMAUD_R", "MDMAUD_F", "MDMAUD_A", 'CLUSTER2', 'GEOCODE2']
ordinal_feats = ['DOMAIN', 'WEALTH1', 'SOLP3', 'SOLIH', 'WEALTH2', "RFA_2", "RFA_3", "RFA_4", "RFA_5", "RFA_6", "RFA_7", "RFA_8", "RFA_9", "RFA_10", "RFA_11", "RFA_12", "RFA_13", "RFA_14", "RFA_15", "RFA_16", "RFA_17", "RFA_18", "RFA_19", "RFA_20", "RFA_21", "RFA_22", "RFA_23", "RFA_24", "RFA_2R", "RFA_2F", "RFA_2A", "MDMAUD_R", "MDMAUD_F", "MDMAUD_A"]
targets = ['TARGET_B', 'TARGET_D']

### Drop unaligned features
This is to drop all features that are not aligned with the variable definitions in the KDD Cup 1998 documention. If they are not aligned (for example where a variable should be binary according to the document, but does not have 2 unique values). No information can be inferred as to what the extra values are, so we drop them.

In [6]:
# Find binary features in dataset
def get_binary_features(df):
    binary_features = []
    for col in df.columns:
        if df[col].nunique() == 2:
            binary_features.append(col)
    return binary_features
found_binary = get_binary_features(df)

In [7]:
# True binary features indicated in KDD CUP 1998 docs
true_binary = [
    'MAILCODE', 'PVASTATE', 'NOEXCH', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'AGEFLAG', 'HOMEOWNR', 'CHILD07', 'CHILD12', 'CHILD18', 'MAJOR', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'PEPSTRFL', 'TARGET_B', 'HPHONE_D'
]

In [8]:
# intersection (features that are true and found binary)
binary_features = list(set(found_binary).intersection(true_binary))

# difference between binary_features and true_binary (features that are true binary but not found)
true_not_found_features = list(set(true_binary).difference(binary_features))

# difference between found_binary and binary_features (features that are found binary but not true)
found_not_true_features = list(set(found_binary).difference(binary_features))

In [9]:
# Drop features that are true binary but not found to be
df.drop(columns=true_not_found_features, inplace=True)
num_feats = [value for value in num_feats if value not in true_not_found_features]
cat_feats = [value for value in cat_feats if value not in true_not_found_features]

### Set proper dtype
Setting dtype to integer for character dtype columns with integers as strings.
ZIP, and CLUSTER codes are maintained (no one-hot encoding due to high cardinality)

*N.B. Only to be run after binary encoding*

In [10]:
# replace blanks with 99 and convert to numeric
df['SOLP3'] = df['SOLP3'].replace(' ', 99).astype(int)
df['SOLIH'] = df['SOLIH'].replace(' ', 99).astype(int)
df['CLUSTER'] = df['CLUSTER'].replace(' ', -1).astype(int)
df['ZIP'] = df['ZIP'].astype(str).str.replace('-', '').astype(int)

### Encoding binary features

In [11]:
# get dictionary of binary features and their encoding
def get_binary_encoding(df, cols):
    binary_encoding = {}
    for col in cols:
        binary_encoding[col] = {df[col].unique()[0]: 0, df[col].unique()[1]: 1}
    return binary_encoding
binary_encoding = get_binary_encoding(df, binary_features)
binary_encoding

{'CDPLAY': {' ': 0, 'Y': 1},
 'HOMEE': {' ': 0, 'Y': 1},
 'CARDS': {' ': 0, 'Y': 1},
 'MAILCODE': {' ': 0, 'B': 1},
 'PETS': {' ': 0, 'Y': 1},
 'STEREO': {' ': 0, 'Y': 1},
 'BOATS': {' ': 0, 'Y': 1},
 'WALKER': {' ': 0, 'Y': 1},
 'PCOWNERS': {' ': 0, 'Y': 1},
 'HPHONE_D': {0: 0, 1: 1},
 'RECSWEEP': {' ': 0, 'X': 1},
 'BIBLE': {' ': 0, 'Y': 1},
 'CATLG': {' ': 0, 'Y': 1},
 'CRAFTS': {' ': 0, 'Y': 1},
 'FISHER': {' ': 0, 'Y': 1},
 'TARGET_B': {0: 0, 1: 1},
 'COLLECT1': {' ': 0, 'Y': 1},
 'RECINHSE': {' ': 0, 'X': 1},
 'PEPSTRFL': {'X': 0, ' ': 1},
 'PLATES': {' ': 0, 'Y': 1},
 'RECP3': {' ': 0, 'X': 1},
 'PHOTO': {' ': 0, 'Y': 1},
 'MAJOR': {' ': 0, 'X': 1},
 'KIDSTUFF': {' ': 0, 'Y': 1},
 'VETERANS': {' ': 0, 'Y': 1},
 'GARDENIN': {' ': 0, 'Y': 1},
 'RECPGVG': {' ': 0, 'X': 1}}

In [12]:
# Binary encoding function
def encode_binary(df, cols, from_to):
    for col in cols:
        encoding = from_to[col]
        df[col] = df[col].replace(encoding)
    return df

In [13]:
# Encode binary features
df = encode_binary(df, binary_features, binary_encoding)

C:\Users\darry\AppData\Local\Temp\ipykernel_12556\1813438623.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(encoding)


### Multiclass categorical feature encoding

First we find the features that are categorical (have dtype object, not all true categorical features). Then we shall handle these features one by one.

In [14]:
# Number of modalities for each feature
def get_categorical_modalities(df):
    categorical_features = []
    for col in df.columns:
        if df[col].dtype == 'object':
            categorical_features.append(col)
    categorical_modalities = {}
    for col in categorical_features:
        categorical_modalities[col] = df[col].nunique()
    return categorical_features, categorical_modalities
categorical_features, categorical_modalities = get_categorical_modalities(df)
categorical_modalities

{'OSOURCE': 896,
 'STATE': 57,
 'MDMAUD': 28,
 'DOMAIN': 17,
 'CHILD03': 4,
 'GENDER': 7,
 'DATASRCE': 4,
 'GEOCODE': 8,
 'LIFESRC': 4,
 'RFA_2': 14,
 'RFA_3': 71,
 'RFA_4': 64,
 'RFA_5': 41,
 'RFA_6': 109,
 'RFA_7': 106,
 'RFA_8': 109,
 'RFA_9': 107,
 'RFA_10': 94,
 'RFA_11': 101,
 'RFA_12': 107,
 'RFA_13': 87,
 'RFA_14': 95,
 'RFA_15': 34,
 'RFA_16': 123,
 'RFA_17': 118,
 'RFA_18': 122,
 'RFA_19': 108,
 'RFA_20': 80,
 'RFA_21': 102,
 'RFA_22': 117,
 'RFA_23': 87,
 'RFA_24': 97,
 'RFA_2R': 1,
 'RFA_2A': 4,
 'MDMAUD_R': 5,
 'MDMAUD_F': 4,
 'MDMAUD_A': 5,
 'GEOCODE2': 5}

*Custom function to bitwise encode RFA, MDMAUD, DOMAIN*

In [15]:
def byte_encoder(code, byte_mapping, true_len, byte_pos):
    if pd.isna(code) or (isinstance(code, str) and len(code) != true_len):
        return np.nan
    elif code == 0 or 'X' in code:
        return 0
    else:
        byte = byte_mapping.get(code[byte_pos])
        return f"{code[:byte_pos]}{byte}{code[byte_pos+1:]}"

*DOMAIN encoding*
1st byte encoding (level of urbanicity):
- R -> 1
- T -> 2
- S -> 3
- C -> 4
- U -> 5

In [16]:
# Code mapping
dom_byte_mapping = {'R': 1, 'T': 2, 'S': 3, 'C': 4, 'U': 5}

df['DOMAIN'] = df['DOMAIN'].apply(byte_encoder, args=(dom_byte_mapping, 2, 0))
df['DOMAIN'] = pd.to_numeric(df['DOMAIN'], errors='coerce').astype('Int64')

*RFA encoding*

Encoding of 1st byte: 
- F -> 1
- N -> 2
- I -> 3
- L -> 5 
- A -> 6
- S -> 7

Encoding of 3rd byte:
- A -> 1
- B -> 3
- C -> 4
- D -> 8
- E -> 13
- F -> 20
- G -> 25

In [17]:
# Code mapping
first_byte_mapping = {'F': 1, 'N': 2, 'I': 3, 'L': 5, 'A': 6, 'S': 7}
third_byte_mapping = {'A': 1, 'B': 3, 'C': 4, 'D': 8, 'E': 13, 'F': 20, 'G': 25}

for col in categorical_features:
    if col.startswith('RFA'):
        if col.endswith('R'):
            df[col] = df[col].apply(byte_encoder, args=(first_byte_mapping, 1, 0))
        elif  col.endswith('A'):
            df[col] = df[col].apply(byte_encoder, args=(third_byte_mapping, 1, 0))
        elif col.endswith('F'):
            df[col] = df[col].replace('X', 0)
        else:
            df[col] = df[col].apply(byte_encoder, args=(first_byte_mapping, 3, 0))
            df[col] = df[col].apply(byte_encoder, args=(third_byte_mapping, 3, 2))
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

*MDMAUD encoding*

If not major donor then 0.

Encoding of 1st byte: 
- D -> 1
- I -> 2
- L -> 3
- C -> 4 

Encoding of 3rd byte:
- L -> 1
- C -> 2
- M -> 3
- T -> 4

In [18]:
# Code mapping
first_byte_mapping = {'D': 1, 'I': 2, 'L': 3, 'C': 4}
third_byte_mapping = {'L': 1, 'C': 2, 'M': 3, 'T': 4}

for col in categorical_features:
    if col.startswith('MDMAUD'):
        if col.endswith('R'):
            df[col] = df[col].apply(byte_encoder, args=(first_byte_mapping, 1, 0))
        elif  col.endswith('A'):
            df[col] = df[col].apply(byte_encoder, args=(third_byte_mapping, 1, 0))
        elif col.endswith('F'):
            df[col] = df[col].replace('X', 0)
        else:
            df[col] = df[col].str[:-1]
            df[col] = df[col].apply(byte_encoder, args=(first_byte_mapping, 3, 0)).apply(byte_encoder, args=(third_byte_mapping, 3, 2))
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

*GENDER*
U -> np.nan
J -> np.nan
M -> 0
F -> 1

In [19]:
# Label encoding
gen_byte_mapping = {np.nan: np.nan, ' ': np.nan, 'U': np.nan, 'J': np.nan, 'M': 0, 'F': 1}
df['GENDER'] = df['GENDER'].apply(lambda x: gen_byte_mapping.get(x))

*GEOCODE, GEOCODE2*
Temporarily label encoded.
One hot encoding (performed at the point of model training using ColumnTransformer in pipeline)
' ' -> np.nan

In [20]:
df['GEOCODE'] = df['GEOCODE'].replace(' ', np.nan).astype('Int64')
geo2_byte_mapping = {np.nan: np.nan, ' ': np.nan, 'A': 1, 'B': 2, 'C': 3, 'D': 4}
df['GEOCODE2'] = df['GEOCODE2'].apply(lambda x: geo2_byte_mapping.get(x)).astype('Int64')

*DATASRCE*
Temporarily label encoded.
One hot encoding (performed at the point of model training using ColumnTransformer in pipeline)
' ' -> np.nan

In [21]:
df['DATASRCE'] = df['DATASRCE'].replace(' ', np.nan).astype('Int64')

*LIFESRC*
Temporarily label encoded.
One hot encoding (performed at the point of model training using ColumnTransformer in pipeline)
' ' -> np.nan

In [22]:
df['LIFESRC'] = df['LIFESRC'].replace(' ', np.nan).astype('Int64')

*CHILD03*
Temporarily label encoded.
One hot encoding, performed at the point of model training using ColumnTransformer in pipeline.
' ' -> np.nan

In [23]:
chil03_byte_mapping = {np.nan: np.nan, ' ': np.nan, 'B': 3, 'M': 2, 'F': 1}
df['CHILD03'] = df['CHILD03'].apply(lambda x: chil03_byte_mapping.get(x))

*OSOURCE*
Label encoding, one hot encoding not possible due to high cardinality.
' ' -> np.nan

In [24]:
df['OSOURCE'] = df['OSOURCE'].replace(' ', np.nan)
osource_encoding = {val: idx for idx, val in enumerate(df['OSOURCE'].unique())}
df['OSOURCE'] = df['OSOURCE'].replace(osource_encoding).astype('Int64')

C:\Users\darry\AppData\Local\Temp\ipykernel_12556\4289099267.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['OSOURCE'] = df['OSOURCE'].replace(osource_encoding).astype('Int64')


*STATE*
Label encoding, one hot encoding not possible due to high cardinality.

In [25]:
df['STATE'] = df['STATE'].replace(' ', np.nan)
osource_encoding = {val: idx for idx, val in enumerate(df['STATE'].unique())}
df['STATE'] = df['STATE'].replace(osource_encoding).astype('Int64')

C:\Users\darry\AppData\Local\Temp\ipykernel_12556\46920350.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['STATE'] = df['STATE'].replace(osource_encoding).astype('Int64')


### Balancing the target variable

In [26]:
df['TARGET_B'].value_counts()

TARGET_B
0    90569
1     4843
Name: count, dtype: int64

In [27]:
df['TARGET_D'].value_counts().head()

TARGET_D
0.0     90569
10.0      941
15.0      591
20.0      577
5.0       503
Name: count, dtype: int64

In [28]:
df_pos = df[df['TARGET_B'] == 1]
df_neg = df[df['TARGET_B'] == 0]

df_neg_sample = df_neg.sample(n=len(df_pos), random_state=42)
df = pd.concat([df_pos, df_neg_sample], axis=0)
len(df)

9686

### Missing value handling

*Step 1: Drop columns with more than 50% missing values*

In [29]:
df.isnull().mean().sort_values(ascending=False).head()

RDATE_5    0.999794
RAMNT_5    0.999794
RAMNT_4    0.995870
RDATE_4    0.995870
RAMNT_3    0.994322
dtype: float64

In [30]:
# Drop columns with more than 50% missing values
threshold = 0.5
df_ = df.dropna(thresh=len(df) * threshold, axis=1)
num_feats = [value for value in num_feats if value in df_.columns]
cat_feats = [value for value in cat_feats if value in df_.columns]
ordinal_feats = [value for value in ordinal_feats if value in df_.columns]
print("Removed columns: " + str(list(set(df.columns).difference(df_.columns))))
df = df_

Removed columns: ['PUBOPP', 'MBCOLECT', 'RAMNT_14', 'RAMNT_5', 'RDATE_16', 'RAMNT_23', 'RDATE_20', 'LIFESRC', 'RAMNT_15', 'RDATE_12', 'RDATE_10', 'RDATE_14', 'RAMNT_11', 'RAMNT_10', 'RAMNT_18', 'RDATE_17', 'RAMNT_22', 'RAMNT_20', 'RAMNT_13', 'RDATE_24', 'RDATE_22', 'RAMNT_24', 'RAMNT_19', 'PUBDOITY', 'RAMNT_3', 'PUBGARDN', 'NUMCHLD', 'RDATE_21', 'MBBOOKS', 'RDATE_6', 'PUBHLTH', 'RAMNT_17', 'MAGFAML', 'RAMNT_9', 'RDATE_4', 'RDATE_8', 'GEOCODE', 'RDATE_13', 'RAMNT_7', 'RAMNT_6', 'RFA_15', 'ADATE_23', 'RDATE_19', 'MBGARDEN', 'RAMNT_12', 'PUBNEWFN', 'MAGFEM', 'RDATE_3', 'RDATE_5', 'PUBPHOTO', 'RDATE_15', 'RAMNT_21', 'MAGMALE', 'RDATE_11', 'RDATE_23', 'RDATE_7', 'ADATE_20', 'RAMNT_16', 'MBCRAFT', 'CHILD03', 'RDATE_18', 'RAMNT_4', 'RAMNT_8', 'RDATE_9', 'RFA_23', 'RFA_20', 'PUBCULIN', 'ADATE_15']


*Step 2: Drop rows with more than 30% missing values*

In [31]:
# recursively sample df_neg so that it does not have more that 30% missing values
def recursive_sample(df_neg, n, threshold):
    sample = df_neg.sample(n)
    temp_ = sample.dropna(thresh=len(df.columns) * threshold, axis=0)
    if n - len(temp_) == 0:
        return sample
    else:
        return recursive_sample(df_neg, n, threshold)

In [32]:
df_pos = df[df['TARGET_B'] == 1]
df_neg = df[df['TARGET_B'] == 0]
df_neg_sample = recursive_sample(df_neg, len(df_pos), 0.7)
df = pd.concat([df_pos, df_neg_sample], axis=0)
len(df)

9686

*Step 3: Impute missing values*

In [33]:
df.isnull().mean().sort_values(ascending=False).head()

WEALTH1     0.471299
WEALTH2     0.448689
RFA_13      0.418233
ADATE_13    0.418129
RFA_21      0.413070
dtype: float64

Impute numerical features

In [34]:
# knn impute numerical features
nimputer = KNNImputer(n_neighbors=10)
df[num_feats] = nimputer.fit_transform(df[num_feats])

Impute categorical features

In [35]:
from fancyimpute import KNN

cimputer = KNN(k=10)
df[ordinal_feats] = cimputer.fit_transform(df[ordinal_feats]).round()

nominal_feats = [value for value in cat_feats if value not in ordinal_feats]
df[nominal_feats] = df[nominal_feats].apply(lambda x: x.fillna(x.mode()[0]))

Imputing row 1/9686 with 4 missing, elapsed time: 20.432
Imputing row 101/9686 with 0 missing, elapsed time: 20.450
Imputing row 201/9686 with 1 missing, elapsed time: 20.474
Imputing row 301/9686 with 10 missing, elapsed time: 20.494
Imputing row 401/9686 with 4 missing, elapsed time: 20.509
Imputing row 501/9686 with 2 missing, elapsed time: 20.531
Imputing row 601/9686 with 0 missing, elapsed time: 20.548
Imputing row 701/9686 with 0 missing, elapsed time: 20.566
Imputing row 801/9686 with 8 missing, elapsed time: 20.585
Imputing row 901/9686 with 4 missing, elapsed time: 20.609
Imputing row 1001/9686 with 11 missing, elapsed time: 20.631
Imputing row 1101/9686 with 0 missing, elapsed time: 20.658
Imputing row 1201/9686 with 2 missing, elapsed time: 20.678
Imputing row 1301/9686 with 1 missing, elapsed time: 20.701
Imputing row 1401/9686 with 2 missing, elapsed time: 20.720
Imputing row 1501/9686 with 1 missing, elapsed time: 20.740
Imputing row 1601/9686 with 18 missing, elapsed ti

In [36]:
print(df.shape)
df.isnull().mean().sort_values(ascending=False).head()

(9686, 406)


ODATEDW    0.0
EC4        0.0
AFC1       0.0
SEC5       0.0
SEC4       0.0
dtype: float64

### Feature engineering

*Custom function to merge subvariables*

In [37]:
def merge_subvars(code):
    if pd.isna(code):
        return np.nan
    elif isinstance(code, str):
        return code
    else:
        bytes = [int(byte) for byte in str(int(code))]
        new = reduce(operator.mul, bytes)
        return new

*DOMAIN*
XY -> X*Y
Both bytes are ordered categorical variables. Instead of separating them we can use combine both their information through multiplication.

In [38]:
df['DOMAIN'] = df['DOMAIN'].apply(merge_subvars)

*RFA*
XYZ -> X*Y*Z
All bytes are ordered categorical variables. Instead of separating them we can use combine both their information through multiplication.

In [39]:
for col in cat_feats:
    if col.startswith('RFA'):
        df[col] = df[col].apply(merge_subvars)

*MDMAUD*
XYZ -> X*Y*Z
All bytes are ordered categorical variables. Instead of separating them we can use combine both their information through multiplication.

In [40]:
for col in categorical_features:
    if col.startswith('MDMAUD'):
        df[col] = df[col].apply(merge_subvars)

### Feature Selection

*Step 1: Manual feature selection*
In this step we manually inspect all features and their meaning and drop features which are not properly described or derived from other features. Due to the large number of features, this is a reasonable first step to reduce the number of features.

Features to remove = ['NOEXCH', 'RECPGVG', 'RECSWEEP', 'CLUSTER', 'AGEFLAG', 'GEOCODE', 'MSA', 'ADI', 'DMA', 'CLUSTER2', 'GEOCODE2']

To confirm that these variables are irrelevant we run some tests.


In [41]:
def sig_tests(df, feats, target, alpha):
    
    keep = []
    
    for feature in feats:
        try:
            x = df[feature]
        except:
            continue

        contingency_table = pd.crosstab(df[feature], df[target])
        
        # Chi-squared test
        chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
        print("Chi-squared test for", feature)
        print("Chi2 statistic:", chi2_stat)
        print("P-value:", p_value)
        if p_value < alpha:
            print("The association between", feature, "and", target, "is significant.")
        else:
            print("The association between", feature, "and", target, "is not significant.")
        
        fisher_p_value = None
        if contingency_table.shape[0] == 2:
            # Fisher's exact test
            odds_ratio, fisher_p_value = fisher_exact(contingency_table)
            print("\nFisher's exact test for", feature)
            print("Odds ratio:", odds_ratio)
            print("P-value:", fisher_p_value)
            if fisher_p_value < alpha:
                print("The association between", feature, "and", target, "is significant.")
            else:
                print("The association between", feature, "and", target, "is not significant.")
        print("\n")
        
        if fisher_p_value is not None:
            if p_value < alpha and fisher_p_value < alpha:
                keep.append(feature)
        else:
            if p_value < alpha:
                keep.append(feature)
        
    return keep
  
      
feats_to_remove = ['RECPGVG', 'RECSWEEP', 'CLUSTER', 'AGEFLAG', 'GEOCODE', 'MSA', 'ADI', 'DMA', 'CLUSTER2', 'GEOCODE2']
target='TARGET_B'
alpha = 0.05
sig_tests(df, feats_to_remove, target, alpha)

Chi-squared test for RECPGVG
Chi2 statistic: 1.2324233194697374
P-value: 0.26693627727899905
The association between RECPGVG and TARGET_B is not significant.

Fisher's exact test for RECPGVG
Odds ratio: 2.252327265204799
P-value: 0.2665211238365676
The association between RECPGVG and TARGET_B is not significant.


Chi-squared test for RECSWEEP
Chi2 statistic: 0.3993586080583721
P-value: 0.5274206843517127
The association between RECSWEEP and TARGET_B is not significant.

Fisher's exact test for RECSWEEP
Odds ratio: 0.8936580819564942
P-value: 0.5275451401703914
The association between RECSWEEP and TARGET_B is not significant.


Chi-squared test for CLUSTER
Chi2 statistic: 79.23031984447691
P-value: 0.01128118954446939
The association between CLUSTER and TARGET_B is significant.


Chi-squared test for MSA
Chi2 statistic: 283.2661746138157
P-value: 0.164251104446483
The association between MSA and TARGET_B is not significant.


Chi-squared test for ADI
Chi2 statistic: 215.44259163099326


['CLUSTER', 'CLUSTER2']

Retained features = ['CLUSTER', 'CLUSTER2']

In [42]:
retained_features = ['CLUSTER', 'CLUSTER2']
feats_to_remove = list(set(feats_to_remove).difference(retained_features))
for feat in feats_to_remove:
    try:
        x = df[feat]
    except:
        feats_to_remove.remove(feat)
df.drop(columns=feats_to_remove, inplace=True)
num_feats = [value for value in num_feats if value not in feats_to_remove]
cat_feats = [value for value in cat_feats if value not in feats_to_remove]
ordinal_feats = [value for value in ordinal_feats if value not in feats_to_remove]

*Step 2: Feature-Target analysis*
Here we test the correlation between features and the target variable, and remove variables that are completely uncorrelated with the target variable.

*Mutual Information analysis*

In [43]:
def get_low_mi_features(df, target_column, threshold, num_feats, cat_feats):

    df_num = df[num_feats]
    df_cat = df[cat_feats]
    df_num_imputed = df_num.fillna(df_num.median())
    df_cat_imputed = df_cat.apply(lambda x: x.fillna(x.mode()[0]))
    df_temp = pd.concat([df_num_imputed, df_cat_imputed, df[target_column]], axis=1)

    for feature in cat_feats:
        if df_temp[feature].dtype == 'object':
            le = LabelEncoder()
            df_temp[feature] = le.fit_transform(df_temp[feature])

    low_mi = []
    
    y = df[target_column].values.ravel()
    
    # For numerical features, compute mutual information
    for feature in (num_feats+cat_feats):
        mi = mutual_info_classif(df_temp[[feature]], y)
        if max(mi) < threshold:
            low_mi.append(feature)

    return low_mi

In [44]:
low_mi = get_low_mi_features(df, 'TARGET_B', 0.1, num_feats, cat_feats)
len(low_mi)

397

*Correlation*
(only numerical features)

In [45]:
def get_low_corr_features(df, target_column, threshold, num_feats, ordinal_feats):

    df_num = df[num_feats]
    df_cat = df[cat_feats]
    df_num_imputed = df_num.fillna(df_num.median())
    df_cat_imputed = df_cat.apply(lambda x: x.fillna(x.mode()[0]))
    df_temp = pd.concat([df_num_imputed, df_cat_imputed, df[target_column]], axis=1)

    low_corr = []
    
    for feature in num_feats+ordinal_feats:
        corr, p_value = spearmanr(df_temp[feature], df_temp[target_column])
        if abs(corr) < threshold:
            low_corr.append(feature)

    return low_corr

low_corr = get_low_corr_features(df, 'TARGET_B', 0.1, num_feats, ordinal_feats)
len(low_corr)

C:\Users\darry\Documents\MDS\Y1_S2\Machine_Learning\mlproject\.venv\Lib\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


355

*PCA Feature Selection*
Numerical features

In [46]:
def perform_pca(data, numerical_vars, n_components=10):
    # Numerical variable preprocessing: Standardization
    scaler = StandardScaler()
    scaled_numerical_data = scaler.fit_transform(data[numerical_vars])

    # Perform PCA for numerical variables
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(scaled_numerical_data)

    # Create DataFrame to store PCA results
    pca_df = pd.DataFrame(pca_result, columns=[f'PC{i}' for i in range(1, n_components+1)], index=df[num_feats].index)

    # Get explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_

    return pca_df, explained_variance_ratio

In [47]:
pca_df, explained_var_pca = perform_pca(df, num_feats, n_components=100)
pca_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100
20,-5.922842,1.022419,-0.036794,0.159332,-1.980363,-3.050329,1.678211,-2.003169,1.223115,-0.754103,...,0.300563,0.836180,-1.075284,-0.543221,0.785543,-1.429821,-0.690517,-0.400406,0.437477,-0.183426
30,-13.856011,-4.163775,3.631028,1.865776,-2.329188,0.757284,-1.300080,-1.617990,1.020627,-2.175827,...,-1.373018,-0.534375,1.427067,-0.039281,-1.004755,-0.325445,-0.595635,0.142939,0.513864,0.487036
45,-6.752256,2.721546,-5.195089,2.002181,-1.468373,-2.251479,2.394843,-0.481035,1.585563,-1.779771,...,-0.186522,1.181914,0.376001,-0.186156,0.835236,0.309851,0.849636,-0.031070,2.237815,-0.262575
78,-9.322682,10.819188,-2.757573,1.759748,3.126652,-0.364349,2.125729,4.606894,0.518054,-1.573970,...,0.889376,-0.968675,0.019127,-0.162733,-0.442054,0.502426,-0.486064,0.183842,0.459358,0.588359
93,-3.002261,-6.128770,6.126433,-2.407620,6.058019,-2.522949,-3.313390,-4.761455,3.739383,-5.963887,...,-0.787302,0.341326,-0.427313,-0.267422,-0.283365,-0.803243,0.598630,0.424904,0.997627,-0.080445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16643,-3.180972,7.933813,8.059284,0.177890,-5.180696,5.883586,3.316556,-0.674838,-1.802726,-0.825500,...,0.011247,-0.774469,-0.176811,-0.364127,0.852660,0.315323,-0.174332,-0.588334,-0.650237,-1.118293
14954,0.060770,12.194021,7.414852,1.070960,0.790588,-2.781871,-1.755778,7.391692,2.143545,2.118790,...,-0.807413,0.445819,1.060798,0.065747,0.517097,-0.061526,-0.476994,-0.066442,-0.040459,0.092263
89609,1.697693,2.904372,-3.367311,-1.474039,-1.469100,-2.389046,0.117179,-0.919052,-1.227924,0.680660,...,-0.689515,-1.427347,0.396265,-0.745791,-0.486365,0.464654,-0.420241,-1.260635,-0.584966,0.549421
51033,-10.903639,1.099832,2.264262,0.012856,8.177562,3.319672,-0.103205,-1.449756,1.749751,3.395480,...,-0.106572,-1.528317,1.404269,-0.679314,-2.186910,0.692113,1.226955,-0.096766,0.755463,-1.461110


In [48]:
# cumulative explained variance
cumulative_explained_var_pca = np.cumsum(explained_var_pca)
cumulative_explained_var_pca

array([0.1193463 , 0.20898918, 0.28516589, 0.32836109, 0.36563576,
       0.39377696, 0.41914037, 0.4408976 , 0.46076937, 0.4785683 ,
       0.49216492, 0.50492217, 0.51705775, 0.52759768, 0.53778181,
       0.54775395, 0.55743309, 0.56618462, 0.57462173, 0.58269297,
       0.59065733, 0.5979196 , 0.60457357, 0.61102881, 0.61732413,
       0.62346376, 0.62933542, 0.63505419, 0.64062781, 0.64600597,
       0.65111591, 0.65617825, 0.6610306 , 0.66571757, 0.67037463,
       0.67491083, 0.67933696, 0.68367008, 0.68794487, 0.69212499,
       0.69623532, 0.70030501, 0.70430425, 0.70824749, 0.71213682,
       0.71592162, 0.71966992, 0.72334408, 0.72697667, 0.73051842,
       0.73403579, 0.73748658, 0.74085289, 0.74417957, 0.7474477 ,
       0.7507074 , 0.7539479 , 0.75714505, 0.76024013, 0.76331573,
       0.76637238, 0.76940377, 0.77238289, 0.77534878, 0.77828372,
       0.78118587, 0.78405986, 0.78691794, 0.78974525, 0.79254543,
       0.79531623, 0.79806721, 0.8007799 , 0.80346632, 0.80612

*FA Feature Selection*
Categorical features

In [49]:
def perform_mca(data, cat_feats, components=10):
    # Perform Factor Analysis for categorical variables
    
    data[cat_feats] = data[cat_feats].astype(float)
    mca = prince.MCA(n_components=components, n_iter=5, copy=True, check_input=True, engine='sklearn', random_state=42)
    mca = mca.fit(data[cat_feats])
    
    # Get the factor loadings and explained variance
    mca_df = mca.transform(data[cat_feats])
    explained_variance = mca.eigenvalues_summary

    return mca_df, explained_variance

mca_df, explained_var_mca = perform_mca(df, cat_feats, components=100)

In [50]:
mca_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
20,1.048690,0.228536,0.168658,0.158912,-0.084861,-0.119968,-0.384367,-0.098970,0.479817,-0.214082,...,-0.287882,0.260797,-0.204004,-0.381016,0.267600,-0.015936,-0.070506,-0.007641,0.071552,0.448481
30,0.498788,-0.038535,0.683448,-0.565573,0.197704,-0.132278,0.449813,-0.374018,0.177333,-0.027086,...,0.024620,0.142893,-0.095052,-0.015910,0.251881,-0.161233,0.051502,-0.345089,-0.219190,-0.119295
45,1.359003,0.512249,0.121986,0.488828,-0.113664,-0.097242,-0.376598,0.043393,0.095280,-0.029880,...,-0.152433,0.344807,-0.266894,0.206027,0.112871,-0.032110,-0.042093,-0.187756,-0.228909,0.163319
78,-0.306378,-0.280970,-0.042793,0.266826,0.124972,-0.218876,-0.095971,-0.176300,-0.197871,0.033635,...,0.171198,0.058418,0.082890,0.089715,0.017284,0.007705,-0.038881,-0.057696,-0.179529,0.155656
93,0.425444,-0.312620,-1.079577,-0.536022,-0.250338,0.223915,0.376358,0.001722,0.118414,-0.071507,...,0.311884,-0.105527,0.300786,0.185191,-0.008182,-0.090964,-0.167153,-0.205256,0.003702,-0.103113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16643,-0.559317,0.978169,-0.019841,-0.208097,-0.277898,-0.394270,0.152333,-0.046885,0.032781,0.050759,...,-0.056579,-0.063399,0.128023,-0.078324,0.129803,0.049378,-0.004302,-0.043959,0.107877,-0.018581
14954,-0.342855,-0.101170,0.138569,0.349925,0.066995,0.245418,0.103416,0.023755,0.033109,-0.068791,...,0.099102,-0.015682,0.129949,-0.042534,-0.131671,0.120141,-0.054222,0.100414,0.056941,0.118492
89609,-0.471029,1.300067,-0.230744,-0.402122,0.815699,0.312745,-0.218936,0.022305,0.229531,0.270705,...,0.504559,-0.088524,0.231609,0.076671,0.009458,-0.039447,0.144756,-0.111505,0.139980,0.343214
51033,-0.538500,0.863327,0.073544,-0.298869,-0.208652,-0.499740,0.119081,0.236066,-0.419676,-0.388627,...,0.024693,-0.089511,-0.008142,0.161183,-0.114418,-0.050632,0.020158,0.218683,-0.156158,0.147069


In [51]:
# cumulative explained variance
explained_var_mca

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.259,0.18%,0.18%
1,0.213,0.15%,0.33%
2,0.191,0.13%,0.46%
3,0.184,0.13%,0.59%
4,0.163,0.11%,0.71%
...,...,...,...
95,0.063,0.04%,6.25%
96,0.063,0.04%,6.30%
97,0.063,0.04%,6.34%


*Significance test feature selection*
Categorical features.

In [61]:
alpha = 0.05
significant = sig_tests(df, cat_feats, 'TARGET_B', alpha)

Chi-squared test for OSOURCE
Chi2 statistic: 584.4030782781556
P-value: 0.008217943329440631
The association between OSOURCE and TARGET_B is significant.


Chi-squared test for TCODE
Chi2 statistic: 49.46793206420607
P-value: 0.014076518271343106
The association between TCODE and TARGET_B is significant.


Chi-squared test for STATE
Chi2 statistic: 79.28310697304653
P-value: 0.00398074295887081
The association between STATE and TARGET_B is significant.


Chi-squared test for ZIP
Chi2 statistic: 5955.88253968254
P-value: 0.15261320591878813
The association between ZIP and TARGET_B is not significant.


Chi-squared test for MAILCODE
Chi2 statistic: 4.525386238261983
P-value: 0.03339553659169869
The association between MAILCODE and TARGET_B is significant.

Fisher's exact test for MAILCODE
Odds ratio: 0.6718007519510337
P-value: 0.0330017582973016
The association between MAILCODE and TARGET_B is significant.


Chi-squared test for RECINHSE
Chi2 statistic: 20.75494473650318
P-value: 5.2196

In [53]:
significant

['OSOURCE',
 'TCODE',
 'STATE',
 'MAILCODE',
 'RECINHSE',
 'RECP3',
 'CLUSTER',
 'WEALTH1',
 'SOLIH',
 'WEALTH2',
 'VETERANS',
 'CRAFTS',
 'WALKER',
 'PEPSTRFL',
 'RFA_2',
 'RFA_3',
 'RFA_4',
 'RFA_5',
 'RFA_6',
 'RFA_7',
 'RFA_8',
 'RFA_9',
 'RFA_10',
 'RFA_11',
 'RFA_12',
 'RFA_13',
 'RFA_14',
 'RFA_16',
 'RFA_17',
 'RFA_18',
 'RFA_19',
 'RFA_21',
 'RFA_22',
 'RFA_24',
 'RFA_2F',
 'RFA_2A',
 'CLUSTER2']

Checking some correlations between suspected informative features and the target.

In [54]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2, _, _, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Calculate Cramer's V for columns A and B
cramers_v_score = cramers_v(df['MAJOR'], df['TARGET_B'])

print("Cramer's V:", cramers_v_score)

Cramer's V: 0.0


### Define the final feature set

In [55]:
df[significant].shape

(9686, 37)

In [56]:
pca_df.iloc[:, :72].shape

(9686, 72)

In [57]:
df_final = pd.concat([df[significant], pca_df.iloc[:, :72]], axis=1)
df_final.shape

(9686, 109)

In [60]:
df_final.head()

,OSOURCE,TCODE,STATE,MAILCODE,RECINHSE,RECP3,CLUSTER,WEALTH1,SOLIH,WEALTH2,...,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72
20,20.0,2.0,10.0,0.0,0.0,0.0,12.0,8.0,99.0,6.0,...,0.359564,-1.605358,0.371449,-1.520905,0.593174,0.024790,-0.769868,0.123167,0.108519,-0.693440
30,22.0,0.0,14.0,0.0,0.0,0.0,35.0,6.0,99.0,5.0,...,-0.892001,-0.235985,-0.193795,-1.098376,-0.247193,-0.942611,0.230213,0.321562,0.699874,1.491970
45,6.0,0.0,7.0,0.0,0.0,0.0,24.0,9.0,99.0,9.0,...,-1.385058,-0.072268,0.812424,0.655906,-1.311188,-0.399538,-0.193921,-1.448552,-1.141853,0.149740
78,54.0,0.0,1.0,0.0,0.0,0.0,13.0,5.0,99.0,9.0,...,-0.073674,-0.245323,0.258453,-0.248076,-0.210410,0.254960,0.273202,-0.354980,1.036938,0.739242
93,23.0,1.0,18.0,0.0,0.0,0.0,18.0,7.0,99.0,7.0,...,0.579192,1.487058,0.471290,-1.052662,1.013979,-0.189355,0.381000,1.433739,0.856298,-0.391180


In [62]:
# save dataframe as csv
#df_final.to_csv('data/preprocessed_data.csv', index=True)